In [ ]:
import igraph as ig
import pandas as pd
import numpy as np
import json
import sqlite3
from sqlite3 import Connection

In [18]:
def build_response_relations(conn: Connection, response_type: str):
    cur = conn.cursor()
    query = """
            SELECT
                t1.author_id as source_author,
                t2.author_id as dest_author
            FROM tweets t1
            JOIN referenced_tweets rt ON t1.id = rt.tweet_id
            JOIN tweets t2 ON rt.referenced_tweet_id = t2.id
            WHERE
                rt.referenced_tweet_type = ?
                AND t1.author_id != t2.author_id
            """
    result = cur.execute(query, [response_type]).fetchall()
    return result

In [21]:
conn = sqlite3.connect(f"D:/UU/Sem3/SDM/social_data_mining/data/twitter.db")
retweet_edges = build_response_relations(conn, "retweeted")
print(len(retweet_edges))

1385201


In [22]:
reply_edges = build_response_relations(conn, "replied_to")
print(len(reply_edges))

38290


In [28]:
def get_distinct_authors(conn: Connection):
    cur = conn.cursor()
    result = cur.execute("SELECT DISTINCT author_id FROM tweets")
    return result.fetchall()

In [34]:
authors = get_distinct_authors(conn)
authors_list = [''.join(author) for author in authors]
# print(authors[:10])
# print(authors_list[:10])
print(len(authors_list))
conn.close()

481467
